In [ ]:
# Preparation

import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description

In [ ]:
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": ["TODO5"],
        "additionalProperties": False
    }
}

In [ ]:
# How should the description for this function look like? Fill in missing parts
# Q: What did you put in TODO3?

# A: query

get_weather_tool = {
    "type": "function",
    "name": "get weather",
    "description": "Retrieves the temperature for a specified city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to retrieve weather data."
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

### Q2. Adding another tool

In [ ]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [ ]:
# Q: Now let's write a description for it. What did you write?

# A:

set_weather_tool = {
    "type": "function",
    "name": "set weather",
    "description": "Sets the temperature for a specified city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to set the weather data."
            },
            "temp": {
                "type": "float",
                "description": "The temperature to associate with the city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

### Q3. Install FastMCP

In [ ]:
%pip install fastmcp

In [1]:
# Q: What's the version of FastMCP you installed?

import fastmcp

print(fastmcp.__version__)

2.10.6


### Q4. Simple MCP Server

In [ ]:
#!python weather_server.py

In [ ]:
# Let's change the example for our case and run it. What do you see in the output?

# Look for a string that matches this template: Starting MCP server 'Demo 🚀' with transport '<TODO>'

# Q: What do you have instead of <TODO>?

# A: stdio

### Q5. Protocol

In [5]:
import subprocess
import json
import threading

# Start the MCP server subprocess (replace `your_server_command` with the actual command)
proc = subprocess.Popen(
    ["python", "weather_server.py"],  # e.g., ["./server"]
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,  # For string I/O
    bufsize=1,  # Line-buffered
)

In [6]:
# Helper to read from stdout asynchronously
def read_stdout(proc):
    while True:
        line = proc.stdout.readline()
        if line:
            print("SERVER RESPONSE:", line.strip())
        else:
            break


# Helper to send JSON-RPC request
def send_request(payload):
    json_line = json.dumps(payload) + "\n"
    proc.stdin.write(json_line)
    proc.stdin.flush()

In [ ]:
# Start the stdout reader in a separate thread
threading.Thread(target=read_stdout, args=(proc,), daemon=True).start()

SERVER RESPONSE: {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.12.0"}}}
SERVER RESPONSE: {"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"get_weather","description":"Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.","inputSchema":{"properties":{"city":{"title":"City","type":"string"}},"required":["city"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"number"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":true}},{"name":"set_weather","description":"Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for whic

In [8]:
# Step 1: Send initialize
send_request({
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2024-11-05",
        "capabilities": {
            "roots": {"listChanged": True},
            "sampling": {}
        },
        "clientInfo": {
            "name": "test-client",
            "version": "1.0.0"
        }
    }
})

# Step 2: Wait a bit for response, then send initialized notification
import time
time.sleep(1)

send_request({
    "jsonrpc": "2.0",
    "method": "notifications/initialized"
})

# Step 3: Ask for list of tools
time.sleep(1)

send_request({
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/list"
})

# Step 4: Wait and watch output to find the tool name for weather (e.g., "weather.get")
# Then call the tool with parameters
time.sleep(2)

send_request({
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/call",
    "params": {
        "name": "get_weather", 
        "arguments": {
            "city": "Berlin"
        }
    }
})

# Keep alive to receive outputs
time.sleep(5)

# Optionally terminate the server
proc.terminate()

In [ ]:
# What did you get in response?

# A: 
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}

### Q6. Client

In [32]:
from fastmcp import Client
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print("Available tools:")
        print(tools)
        print()
        print('-----------')
        for tool in tools:
            print(f"- {tool.name}: {tool.description}")
            print()
        
        print()
        result = await mcp_client.call_tool("get_weather", {"city": 'Berlin'})
        print(f"Result: {result.content}")

# Run it in Jupyter
await main()

Available tools:
[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'tem

In [ ]:
# Q: Use the client to get the list of available tools of our script. How does the result look like?

# A: 
[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, outputSchema=None, annotations=None, meta=None)]

<coroutine object main at 0x790eb4713680>